In [1]:

from functions.adjust_cases_functions import prepare_cases
from seir_model import SEIRD

import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import numpy as np
import os

from global_config import config


data_dir            = config.get_property('data_dir_covid')
geo_dir             = config.get_property('geo_dir')
data_dir_mnps       = config.get_property('data_dir_col')
results_dir         = config.get_property('results_dir')
agglomeration_df    = pd.read_csv(os.path.join(data_dir_mnps, 'administrative_division_col_2018.csv')).set_index('poly_id')
agglomerated_folder = os.path.join(data_dir, 'data_stages', 'colombia', 'agglomerated', 'geometry' )

data =  pd.read_csv(os.path.join(agglomerated_folder, 'cases.csv'), parse_dates=['date_time'], dayfirst=True).set_index('poly_id').loc[11001].set_index('date_time')
data = data.resample('D').sum().fillna(0)[['num_cases','num_diseased']]
data  = prepare_cases(data, col='num_cases', cutoff=0)    # .rename({'smoothed_num_cases':'num_cases'})
data  = prepare_cases(data, col='num_diseased', cutoff=0) # .rename({'smoothed_num_cases':'num_cases'})

data = data.rename(columns={'smoothed_num_cases': 'confirmed', 'smoothed_num_diseased':'death'})[['confirmed', 'death']]


data = data.iloc[:-11]

In [2]:
data

,confirmed,death
date_time,,
2020-03-06,0.0,0.0
2020-03-07,0.0,0.0
2020-03-08,0.0,0.0
2020-03-09,0.0,0.0
2020-03-10,1.0,0.0
...,...,...
2021-01-15,5489.0,79.0
2021-01-16,5217.0,71.0
2021-01-17,5025.0,64.0


In [3]:
import jax
import rpy2
import numpy as np
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
splines = importr("splines")
predict = importr("stats")


In [ ]:
df_full = pd.DataFrame({'X':np.arange(len(data.valuecases.tolist())),
                   'Y':np.array(data.confirmed.tolist()),  # add your data here
                    'D': np.array(data.death.tolist())})
                  
df_train =df_full

knots=np.arange(0,len(df_full.X.values)-10,10)
knots = np.concatenate((knots,[len(df_full.X.values)+28]))

#knots= np.concatenate((knots[:-1],[len(df_full.X.values)+28]))
#print (knots)
basis = splines.bs(np.arange(0,len(df_full.X.values)+28),degree=3,knots=knots)


basis_matrix = np.array(basis)
basis_train= basis_matrix[:len(df_full.X.values),:]

#basis_oos = predict.predict(basis,newx=np.arange(1,(len(df_full.X.values)+29)))
basis_oos_matrix =basis_matrix
num_data =basis_matrix.shape[0]
num_basis = basis_matrix.shape[1]




def ExponentialRandomWalk(loc=1., scale=1e-2, drift=0., num_steps=100):
    '''
    Return distrubtion of exponentiated Gaussian random walk
    
    Variables are x_0, ..., x_{T-1}
    
    Dynamics in log-space are random walk with drift:
       log(x_0) := log(loc) 
       log(x_t) := log(x_{t-1}) + drift + eps_t,    eps_t ~ N(0, scale)
        
    ==> Dynamics in non-log space are:
        x_0 := loc
        x_t := x_{t-1} * exp(drift + eps_t),    eps_t ~ N(0, scale)        
    '''
    
    log_loc = np.log(loc) + drift * (np.arange(num_steps)+0.)
    
    return dist.TransformedDistribution(
        dist.GaussianRandomWalk(scale=scale, num_steps=num_steps),
        [
            dist.transforms.AffineTransform(loc = log_loc, scale=1.),
            dist.transforms.ExpTransform()
        ]
    )

def LogisticRandomWalk(loc=1., scale=1e-2, drift=0., num_steps=100):
    '''
    Return distrubtion of exponentiated Gaussian random walk
   
    Variables are x_0, ..., x_{T-1}
   
    Dynamics in log-space are random walk with drift:
       log(x_0) := log(loc)
       log(x_t) := log(x_{t-1}) + drift + eps_t,    eps_t ~ N(0, scale)
    
    ==> Dynamics in non-log space are:
        x_0 := loc
        x_t := x_{t-1} * exp(drift + eps_t),    eps_t ~ N(0, scale)
    '''
   
    logistic_loc = np.log(loc/(1-loc)) + drift * (np.arange(num_steps)+0.)
   
    return dist.TransformedDistribution(
        dist.GaussianRandomWalk(scale=scale, num_steps=num_steps),
        [
            dist.transforms.AffineTransform(loc = logistic_loc, scale=1.),
            dist.transforms.SigmoidTransform()
        ]
    )